# SigLIP Feature Extraction and Gradient Boosting Ensemble Training Pipeline

I'll use a hybrid approach by combining "frozen" deep learning embeddings from a pre-trained Vision-Language model (SigLIP) with traditional statistical machine learning models (Gradient Boosting) to predict different types of dry biomass (Clover, Green, Dead, Total, etc.).

In [1]:
import os
import gc
import random
import pickle
from pathlib import Path
from dataclasses import dataclass
from copy import deepcopy

import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm

import torch
from PIL import Image
from transformers import AutoModel, AutoImageProcessor, AutoTokenizer

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import HistGradientBoostingRegressor, GradientBoostingRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import r2_score
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

import warnings
warnings.filterwarnings("ignore")
os.environ["TOKENIZERS_PARALLELISM"] = "false"

2026-02-04 13:10:27.160075: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770210627.353654      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770210627.409086      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770210627.870268      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770210627.870309      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770210627.870312      55 computation_placer.cc:177] computation placer alr

In [2]:
@dataclass
class Config:
    DATA_PATH: Path = Path("/kaggle/input/csiro-biomass/")
    SIGLIP_PATH: str = "/kaggle/input/google-siglip-so400m-patch14-384/transformers/default/1"
    OUTPUT_DIR: Path = Path("./siglip_models")
    
    SEED: int = 42
    DEVICE: str = "cuda" if torch.cuda.is_available() else "cpu"
    
    PATCH_SIZE: int = 520
    OVERLAP: int = 16
    N_FOLDS: int = 5
    
    TARGET_NAMES = ['Dry_Clover_g', 'Dry_Dead_g', 'Dry_Green_g', 'Dry_Total_g', 'GDM_g']
    TRAIN_TARGETS = ['Dry_Dead_g', 'Dry_Green_g', 'Dry_Total_g', 'GDM_g']  # Skip Clover (always 0)
    
    TARGET_MAX = {
        "Dry_Clover_g": 71.7865,
        "Dry_Dead_g": 83.8407,
        "Dry_Green_g": 157.9836,
        "Dry_Total_g": 185.70,
        "GDM_g": 157.9836,
    }


cfg = Config()
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

In [3]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


set_seed(cfg.SEED)
print(f"Device: {cfg.DEVICE}")
print(f"Output dir: {cfg.OUTPUT_DIR}")

Device: cuda
Output dir: siglip_models


In [4]:
def pivot_table(df: pd.DataFrame) -> pd.DataFrame:
    
    df_pt = pd.pivot_table(
        df, 
        values='target', 
        index=['image_path', 'Sampling_Date', 'State', 'Species', 'Pre_GSHH_NDVI', 'Height_Ave_cm'], 
        columns='target_name', 
        aggfunc='mean'
    ).reset_index()

    return df_pt


In [5]:
def get_robust_stratified_folds(df, n_splits=5, seed=42):

    df = df.copy().reset_index(drop=True)

    # Feature Engineering
    
    # qcut: sort all the biomass values and split them into three equally sized groups
    df['bin_total'] = pd.qcut(df['Dry_Total_g'], q=3, labels=["L", "M", "H"])

    living_mass = df['Dry_Clover_g'] + df['Dry_Green_g']
    # calculate what percentage of living plant matter is clover
    df['clover_frac'] = df['Dry_Clover_g'] / (living_mass + 1e-6)

    # manual boundaries of low: 0-20%, high: 20-100% to ensuere that we don't put all clover rich samples in one fold
    df['bin_clover'] = pd.cut(df['clover_frac'], bins=[-0.1, 0.2, 1.1], labels=["Lo", "Hi"])

    df['state_key'] = df['State'].astype(str) #covert into str so we can combine it later

    # Define Hierarchy Keys
    df['key_L1'] = df['state_key'] + "_" + df['bin_total'].astype(str) + "_" + df['bin_clover'].astype(str) # dream scenario - balance everything
    df['key_L2'] = df['state_key'] + "_" + df['bin_total'].astype(str) # just try state and total mass
    df['key_L3'] = df['state_key']

    # for robustness - check if the groups are big enough >= 5 samples
    
    df['final_stratify'] = df['key_L1']
    counts = df['final_stratify'].value_counts()
    rare_keys = counts[counts < n_splits].index

    #overwrite those keys with a simpler state and total mass key like WA_Hi
    mask_rare_L1 = df['final_stratify'].isin(rare_keys) # panda series (boolean)
    df.loc[mask_rare_L1, 'final_stratify'] = df.loc[mask_rare_L1, 'key_L2']

    #check again and downgrade to L3 if needed
    counts = df['final_stratify'].value_counts()
    rare_keys = counts[counts < n_splits].index
    mask_rare_L2 = df['final_stratify'].isin(rare_keys)
    df.loc[mask_rare_L2, 'final_stratify'] = df.loc[mask_rare_L2, 'key_L3']

    #final safety net
    counts = df['final_stratify'].value_counts()
    rare_keys = counts[counts < n_splits].index
    if len(rare_keys) > 0:
        print(f"Warning: Dropping {len(rare_keys)} extremely rare buckets to 'Misc'")
        df.loc[df['final_stratify'].isin(rare_keys), 'final_stratify'] = 'Misc'

    # Perform Split
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    
    df['fold'] = -1
    for fold, (train_idx, val_idx) in enumerate(skf.split(df, df['final_stratify'])):
        df.loc[val_idx, 'fold'] = fold
        
    return df

In [6]:
#Feature Extraction

def split_image(image, patch_size=520, overlap=16):
    """
    Cut the large image into smaller squares so the model can look at them one by one.
    Takes the image, the desired square size (520px), and how much the squares should overlap (16px).
    """
    h, w, c = image.shape
    stride = patch_size - overlap #step size - move 504px for the next patch instead of 520px to include 16px of previous patch(context)
    patches = []
    
    for y in range(0, h, stride):
        for x in range(0, w, stride):
            y2 = min(y + patch_size, h)
            x2 = min(x + patch_size, w)
            y1 = max(0, y2 - patch_size)
            x1 = max(0, x2 - patch_size)
            patch = image[y1:y2, x1:x2, :]
            patches.append(patch)
            
    return patches

def compute_embeddings(model_path, df):
    """ 
    Loads the SigLIP model and loops through every single image and converts it into a list of numbers (embedding).
    """
    print(f"Computing Embeddings for {len(df)} images...")
    model = AutoModel.from_pretrained(model_path, local_files_only=True).eval().to(cfg.DEVICE)
    processor = AutoImageProcessor.from_pretrained(model_path, local_files_only=True)
    
    EMBEDDINGS = []
    
    for _, row in tqdm(df.iterrows(), total=len(df)):
        try:
            img = cv2.imread(row['image_path']) #converts image into a grid of numbers from 0-255
            
            if img is None:
                raise ValueError("Image not found")
                
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            patches = split_image(img, patch_size=cfg.PATCH_SIZE, overlap=cfg.OVERLAP)
            #convert patches from numpy arrays to PIL images because that's what HF processor prefers as input.
            images = [Image.fromarray(p) for p in patches]
            
            inputs = processor(images=images, return_tensors="pt").to(cfg.DEVICE) #converts numbers from opencv into tensors
            with torch.no_grad():
                features = model.get_image_features(**inputs) #look at a batch of patches and output a vector for each
            
            # take the average of all patches into a single vector to represent an image.
            avg_embed = features.mean(dim=0).cpu().numpy()
            EMBEDDINGS.append(avg_embed)
            
        except Exception as e:
            
            print(f"Error processing {row['image_path']}: {e}")
            EMBEDDINGS.append(np.zeros(1152))
        
    torch.cuda.empty_cache()
    del model
    gc.collect()
    return np.stack(EMBEDDINGS)

In [7]:
def generate_semantic_features(image_embeddings_np, model_path):
    
    print("Generating Semantic Features...")
    model = AutoModel.from_pretrained(model_path, local_files_only=True).to(cfg.DEVICE)
    tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)
    
    concept_groups = {
        "bare": ["bare soil", "dirt ground", "sparse vegetation", "exposed earth"],    #average the results of 4 similar phrases to get an accurate signal
        "sparse": ["low density pasture", "thin grass", "short clipped grass"],
        "medium": ["average pasture cover", "medium height grass", "grazed pasture"],
        "dense": ["dense tall pasture", "thick grassy volume", "high biomass", "overgrown vegetation"],
        "green": ["lush green vibrant pasture", "photosynthesizing leaves", "fresh growth"],
        "dead": ["dry brown dead grass", "yellow straw", "senesced material", "standing hay"],
        "clover": ["white clover", "trifolium repens", "broadleaf legume", "clover flowers"],
        "grass": ["ryegrass", "blade-like leaves", "fescue", "grassy sward"]
    }
    
    concept_vectors = {}
    with torch.no_grad():
        for name, prompts in concept_groups.items():
            inputs = tokenizer(prompts, padding="max_length", return_tensors="pt").to(cfg.DEVICE)
            emb = model.get_text_features(**inputs) #results in a matrix of shape (4, 1152) -> 4 phrases and 1152 features.
            emb = emb / emb.norm(p=2, dim=-1, keepdim=True) # a / |a|
            concept_vectors[name] = emb.mean(dim=0, keepdim=True) #use the average of the 4 phrase vectors to create 1 master vector
            
    img_tensor = torch.tensor(image_embeddings_np, dtype=torch.float32).to(cfg.DEVICE)
    img_tensor = img_tensor / img_tensor.norm(p=2, dim=-1, keepdim=True) #b / |b|

    #both image and phrase vectors represent a single point in a 1152 dimensional space
    #length of both vectors is 1 because they are normalized
    scores = {}
    for name, vec in concept_vectors.items():
        #use geometric interpretation of dot product to get the scores that represent the cosine similarity between the 2 vectors.
        #the scores are on a range of 0-1. cosine similarity or cos(theta) = a * b / |a| * |b|
        scores[name] = torch.matmul(img_tensor, vec.T).cpu().numpy().flatten() #results in a list of scores for each image which becomes value in a dict
        
    df_scores = pd.DataFrame(scores)
    df_scores['ratio_greenness'] = df_scores['green'] / (df_scores['green'] + df_scores['dead'] + 1e-6) #what percentage of grass is alive
    df_scores['ratio_clover'] = df_scores['clover'] / (df_scores['clover'] + df_scores['grass'] + 1e-6)
    #meassure density (ground clover). this is the strongest predictor of biomass
    df_scores['ratio_cover'] = (df_scores['dense'] + df_scores['medium']) / (df_scores['bare'] + df_scores['sparse'] + 1e-6)
    
    torch.cuda.empty_cache()
    del model
    gc.collect()
    return df_scores.values

In [8]:
class SupervisedEmbeddingEngine:
    """
    Custom Feature Engineering class that compresses the noisy input (1,152 columns) into a small, highly potent set of features (approx. 30-40 columns).
    It combines three different mathematical techniques (PCA, PLS, GMM) into one pipeline.
    """
    def __init__(self, n_pca=0.80, n_pls=8, n_gmm=6, random_state=42):
        self.scaler = StandardScaler()
        #Unsupervised Compression. It will condense the data until it retains 80% of the original information
        self.pca = PCA(n_components=n_pca, random_state=random_state)
        #Supervised Extraction. This is the smart tool. It looks for 8 specific patterns that correlate with Target (Biomass).
        self.pls = PLSRegression(n_components=n_pls, scale=False)
        #Clustering. It will try to sort every image into one of 6 types
        self.gmm = GaussianMixture(n_components=n_gmm, covariance_type='diag', random_state=random_state)
        self.pls_fitted_ = False

    # the training - fit function is the learner. It calculates math rules based on the data you give it. It just saves these numbers in its internal memory.
    def fit(self, X, y=None, X_semantic=None):
        # Scale
        X_scaled = self.scaler.fit_transform(X) #It calculates the mean and s.d of every column in training set and stores it. Then, it standardizes X
        
        # Unsupervised - train pca and gmm
        self.pca.fit(X_scaled)
        self.gmm.fit(X_scaled)
        
        # Supervised
        if y is not None:
            self.pls.fit(X_scaled, y) #learn relationships between image features and bimoass targets
            self.pls_fitted_ = True
        return self

    #the execution - it applies the saved rules in fit function to modify the data
    def transform(self, X, X_semantic=None):
        X_scaled = self.scaler.transform(X) #standardizes the data based on the old mean and s.d. we learned in fit

        #It performs a matrix multiplication between scaled data and the PCA Eigenvectors (the projection matrix) stored inside self.pca.
        #The Result: A matrix of roughly 25-30 columns capturing the "General Structure" of the images.
        features = [self.pca.transform(X_scaled)]
        
        if self.pls_fitted_:
            features.append(self.pls.transform(X_scaled))  #matrix multiplication with pls weights
            
        features.append(self.gmm.predict_proba(X_scaled))
        
        if X_semantic is not None:
            # Normalize using INPUT's own mean/std
            sem_norm = (X_semantic - np.mean(X_semantic, axis=0)) / (np.std(X_semantic, axis=0) + 1e-6)
            features.append(sem_norm)
            
        return np.hstack(features)

In [9]:
# MODEL CONFIGURATIONS
MODEL_CONFIGS = {
    'hist': {
        'class': HistGradientBoostingRegressor,
        'params': {
            'max_iter': 300, 'learning_rate': 0.05, 'max_depth': None,
            'l2_regularization': 0.44, 'random_state': 42
        }
    },
    'gb': {
        'class': GradientBoostingRegressor,
        'params': {
            'n_estimators': 1354, 'learning_rate': 0.010, 'max_depth': 3,
            'subsample': 0.60, 'random_state': 42
        }
    },
    'cat': {
        'class': CatBoostRegressor,
        'params': {
            'iterations': 1900, 'learning_rate': 0.045, 'depth': 4, 'l2_leaf_reg': 0.56,
            'random_strength': 0.045, 'bagging_temperature': 0.98, 'verbose': 0,
            'random_state': 42, 'allow_writing_files': False
        }
    },
    'lgbm': {
        'class': LGBMRegressor,
        'params': {
            'n_estimators': 807, 'learning_rate': 0.014, 'num_leaves': 48,
            'min_child_samples': 19, 'subsample': 0.745, 'colsample_bytree': 0.745,
            'reg_alpha': 0.21, 'reg_lambda': 3.78, 'verbose': -1, 'random_state': 42
        }
    }
}

In [10]:
def train_and_save_models():
    print("\n" + "="*80)
    print("LOADING DATA")
    print("="*80)
    
    # Load train.csv and pivot (matches original)
    train_long = pd.read_csv(cfg.DATA_PATH / "train.csv")
    train_df = pivot_table(train_long)
    
    # Fill any missing targets
    for t in cfg.TARGET_NAMES:
        if t not in train_df.columns:
            train_df[t] = 0.0
    train_df[cfg.TARGET_NAMES] = train_df[cfg.TARGET_NAMES].fillna(0.0)
    
    # Create folds
    train_df = get_robust_stratified_folds(train_df, n_splits=cfg.N_FOLDS, seed=cfg.SEED)
    
    # Fix paths (matches original)
    if not str(train_df['image_path'].iloc[0]).startswith('/'):
        train_df['image_path'] = train_df['image_path'].apply(
            lambda p: str(cfg.DATA_PATH / 'train' / os.path.basename(p))
        )
    
    print(f"Train samples: {len(train_df)}")
    print(f"Fold distribution:\n{train_df['fold'].value_counts().sort_index()}")
    
    # Compute embeddings
    print("\n" + "="*80)
    print("COMPUTING EMBEDDINGS")
    print("="*80)
    
    train_embeddings = compute_embeddings(cfg.SIGLIP_PATH, train_df)
    
    # Create Feature DataFrame
    emb_cols = [f"emb{i}" for i in range(train_embeddings.shape[1])]
    train_feat_df = pd.concat([train_df, pd.DataFrame(train_embeddings, columns=emb_cols)], axis=1)
    
    print(f"Train Features Shape: {train_feat_df.shape}")
    
    # Generate semantic features
    sem_train = generate_semantic_features(train_embeddings, cfg.SIGLIP_PATH)
    print(f"Semantic Features Shape: {sem_train.shape}")
    
    # Prepare data
    target_max_arr = np.array([cfg.TARGET_MAX[t] for t in cfg.TARGET_NAMES], dtype=np.float32)
    
    X_train_full = train_feat_df[emb_cols].values.astype(np.float32) #shape - (N images, 1152)
    y_train_full = train_feat_df[cfg.TARGET_NAMES].values.astype(np.float32) #shape - (N images, 5)
    
    # Storage
    all_engines = {}
    all_models = {}
    
    print("\n" + "="*80)
    print("TRAINING MODELS")
    print("="*80)

    # cross validation process. Train multiple models on different subsets of the data.
    for fold in range(cfg.N_FOLDS):
        print(f"\n--- Fold {fold} ---")
        
        #use images not in the fold for training
        train_mask = train_feat_df['fold'] != fold
        
        X_tr = X_train_full[train_mask]
        #normalize the targets to 0-1 by dividing using max observed value for that target
        y_tr = y_train_full[train_mask] / target_max_arr
        sem_tr_fold = sem_train[train_mask]
        
        # Fit feature engine (matches original)
        engine = SupervisedEmbeddingEngine(n_pca=0.80, n_pls=8, n_gmm=6, random_state=cfg.SEED)
        engine.fit(X_tr, y=y_tr, X_semantic=sem_tr_fold)
        all_engines[fold] = engine
        
        # Transform training data
        x_tr_eng = engine.transform(X_tr, X_semantic=sem_tr_fold)
        
        # Train each model type
        all_models[fold] = {}
        
        for model_name, model_cfg in MODEL_CONFIGS.items():
            print(f"  Training {model_name}...")
            all_models[fold][model_name] = {}

            #standard regression models usually only predict one number at a time. so we predict each biomass type seperately.
            for target_name in cfg.TRAIN_TARGETS:
                k = cfg.TARGET_NAMES.index(target_name)
                
                #creates a fresh untrained model ready to learn
                model = model_cfg['class'](**model_cfg['params'])
                model.fit(x_tr_eng, y_tr[:, k])
                #save the brain
                all_models[fold][model_name][target_name] = model
        
        print(f"  Fold {fold} complete.")
    
    # Save everything
    print("\n" + "="*80)
    print("SAVING MODELS")
    print("="*80)
    
    with open(cfg.OUTPUT_DIR / "feature_engines.pkl", "wb") as f:
        pickle.dump(all_engines, f)
    print(f"Saved feature_engines.pkl ({cfg.N_FOLDS} engines)")
    
    with open(cfg.OUTPUT_DIR / "models.pkl", "wb") as f:
        pickle.dump(all_models, f)
    print(f"Saved models.pkl ({cfg.N_FOLDS} folds × {len(MODEL_CONFIGS)} models × {len(cfg.TRAIN_TARGETS)} targets)")
    
    # Save config
    config_dict = {
        'TARGET_NAMES': cfg.TARGET_NAMES,
        'TRAIN_TARGETS': cfg.TRAIN_TARGETS,
        'TARGET_MAX': cfg.TARGET_MAX,
        'N_FOLDS': cfg.N_FOLDS,
        'SEED': cfg.SEED,
        'PATCH_SIZE': cfg.PATCH_SIZE,
        'OVERLAP': cfg.OVERLAP,
        'emb_cols': emb_cols,
        'model_names': list(MODEL_CONFIGS.keys()),
    }
    with open(cfg.OUTPUT_DIR / "config.pkl", "wb") as f:
        pickle.dump(config_dict, f)
    print("Saved config.pkl")
    
if __name__ == "__main__":
    train_and_save_models()


LOADING DATA
Train samples: 357
Fold distribution:
fold
0    72
1    72
2    71
3    71
4    71
Name: count, dtype: int64

COMPUTING EMBEDDINGS
Computing Embeddings for 357 images...


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
100%|██████████| 357/357 [06:32<00:00,  1.10s/it]


Train Features Shape: (357, 1172)
Generating Semantic Features...
Semantic Features Shape: (357, 11)

TRAINING MODELS

--- Fold 0 ---
  Training hist...
  Training gb...
  Training cat...
  Training lgbm...
  Fold 0 complete.

--- Fold 1 ---
  Training hist...
  Training gb...
  Training cat...
  Training lgbm...
  Fold 1 complete.

--- Fold 2 ---
  Training hist...
  Training gb...
  Training cat...
  Training lgbm...
  Fold 2 complete.

--- Fold 3 ---
  Training hist...
  Training gb...
  Training cat...
  Training lgbm...
  Fold 3 complete.

--- Fold 4 ---
  Training hist...
  Training gb...
  Training cat...
  Training lgbm...
  Fold 4 complete.

SAVING MODELS
Saved feature_engines.pkl (5 engines)
Saved models.pkl (5 folds × 4 models × 4 targets)
Saved config.pkl
